# TD - Graph Traversal: BFS, DFS and Shortest Paths

## Objectives
- Understand and implement breadth-first search (BFS)
- Understand and implement depth-first search (DFS)
- Use shortest path algorithms
- Visualize results with Plotly on a real graph (Paris road network)

## Data
We will use the Paris road network obtained via OSMnx, which retrieves OpenStreetMap data.


In [ ]:
# Installation of necessary libraries
# !pip install osmnx networkx plotly matplotlib numpy

In [ ]:
import osmnx as ox
import networkx as nx
import plotly.graph_objects as go
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from IPython.display import display, HTML

print("Libraries imported successfully!")
print(f"NetworkX version: {nx.__version__}")
print(f"OSMnx version: {ox.__version__}")

## 1. Loading the Paris Road Network Graph


In [ ]:
# Loading the Paris road network graph
print("Loading Paris road network...")
G = ox.graph_from_place("Paris, France", network_type="drive")

# Convert to undirected NetworkX graph to simplify algorithms
# (we can keep the directed graph if necessary)
G = G.to_undirected()

print(f"Graph loaded: {len(G.nodes())} nodes, {len(G.edges())} edges")
print(f"Graph type: {type(G)}")
print(f"Is connected: {nx.is_connected(G)}")

In [ ]:
# Select a main connected component if necessary
if not nx.is_connected(G):
    print("Graph is not connected. Selecting the largest connected component...")
    largest_cc = max(nx.connected_components(G), key=len)
    G = G.subgraph(largest_cc).copy()
    print(f"New graph: {len(G.nodes())} nodes, {len(G.edges())} edges")

# Select two nodes for our experiments
nodes = list(G.nodes())
start_node = nodes[0]
end_node = nodes[len(nodes)//4]  # A node at approximately 1/4 of the graph

print(f"\nStart node: {start_node}")
print(f"End node: {end_node}")

## 2. Utility Function for Visualization with Plotly


In [ ]:
def plot_graph_plotly(G, title="Graph", highlight_nodes=None, highlight_path=None, node_colors=None):
    """
    Visualize a graph with Plotly on a map
    
    Parameters:
    - G: NetworkX graph
    - title: chart title
    - highlight_nodes: list of nodes to highlight
    - highlight_path: list of nodes forming a path to highlight
    - node_colors: dictionary {node: color} to color nodes
    """
    # Get coordinates (longitude, latitude)
    pos = {}
    for node in G.nodes():
        if 'x' in G.nodes[node] and 'y' in G.nodes[node]:
            # OSMnx stores x=longitude, y=latitude
            pos[node] = (G.nodes[node]['x'], G.nodes[node]['y'])
        elif 'lon' in G.nodes[node] and 'lat' in G.nodes[node]:
            pos[node] = (G.nodes[node]['lon'], G.nodes[node]['lat'])
        else:
            # Fallback: use node ID as coordinates
            pos[node] = (node, 0)
    
    # Calculate map center
    if pos:
        lons = [p[0] for p in pos.values() if isinstance(p[0], (int, float))]
        lats = [p[1] for p in pos.values() if isinstance(p[1], (int, float))]
        if lons and lats:
            center_lon = sum(lons) / len(lons)
            center_lat = sum(lats) / len(lats)
        else:
            center_lon, center_lat = 2.3522, 48.8566  # Paris by default
    else:
        center_lon, center_lat = 2.3522, 48.8566
    
    fig = go.Figure()
    
    # Prepare edges with coordinates (lon, lat)
    for edge in G.edges():
        if edge[0] in pos and edge[1] in pos:
            lon0, lat0 = pos[edge[0]]
            lon1, lat1 = pos[edge[1]]
            # Check that coordinates are valid
            if (isinstance(lon0, (int, float)) and isinstance(lat0, (int, float)) and
                isinstance(lon1, (int, float)) and isinstance(lat1, (int, float))):
                fig.add_trace(go.Scattermap(
                    mode='lines',
                    lon=[lon0, lon1],
                    lat=[lat0, lat1],
                    line=dict(width=0.5, color='#888'),
                    hoverinfo='none',
                    showlegend=False
                ))
    
    # Prepare nodes
    node_lons = []
    node_lats = []
    node_text = []
    node_colors_list = []
    node_sizes = []
    
    for node in G.nodes():
        if node in pos:
            lon, lat = pos[node]
            if isinstance(lon, (int, float)) and isinstance(lat, (int, float)):
                node_lons.append(lon)
                node_lats.append(lat)
                node_text.append(f'Node {node}')
                
                # Determine color and size
                if highlight_path and node in highlight_path:
                    color = 'red'
                    size = 10
                elif highlight_nodes and node in highlight_nodes:
                    color = 'orange'
                    size = 12
                elif node_colors and node in node_colors:
                    color = node_colors[node]
                    size = 8
                else:
                    color = 'lightblue'
                    size = 4
                node_colors_list.append(color)
                node_sizes.append(size)
    
    # Add trace for nodes
    if node_lons and node_lats:
        fig.add_trace(go.Scattermap(
            mode='markers',
            lon=node_lons,
            lat=node_lats,
            text=node_text,
            hoverinfo='text',
            marker=dict(
                size=node_sizes,
                color=node_colors_list,
                opacity=0.8
            ),
            showlegend=False
        ))
    
    # Update layout with mapbox
    fig.update_layout(
        title=title,
        mapbox=dict(
            style="open-street-map",  # Use OpenStreetMap as basemap
            center=dict(lon=center_lon, lat=center_lat),
            zoom=11,  # Zoom level adapted to Paris
            bearing=0,
            pitch=0
        ),
        margin=dict(l=0, r=0, t=40, b=0),
        height=700
    )
    
    return fig

## 3. Initial Graph Visualization


In [ ]:
# Visualization of the complete graph (sampled for performance)
sample_nodes = list(G.nodes())[::10]  # Take 1 node out of 10 for visualization
G_sample = G.subgraph(sample_nodes).copy()

fig = plot_graph_plotly(G_sample, title="Paris Road Network (sample)")
fig.show()

## 4. Breadth-First Search (BFS)

### 4.0 BFS Theory

**Breadth-First Search (BFS)** is a fundamental graph traversal algorithm that systematically explores a graph level by level.

#### Principle of Operation

BFS uses a **queue (FIFO: First In First Out)** to maintain the order of node visits:

1. **Initialization**: Start with a source node, mark it as visited and add it to the queue
2. **Exploration**: While the queue is not empty:
   - Remove the first node from the queue (dequeue)
   - Visit all its unvisited neighbors
   - Mark these neighbors as visited and add them to the end of the queue (enqueue)
3. **Result**: We obtain the order of visits level by level

#### Important Properties

- **Shortest path**: On an **unweighted** graph, BFS guarantees finding the shortest path in number of edges between two nodes
- **Data structure**: Queue (FIFO) - the first node added is the first to be processed
- **Time complexity**: O(V + E) where V is the number of nodes and E is the number of edges
- **Space complexity**: O(V) to store the queue and visited nodes

#### Applications

- Maze navigation
- Social networks: finding the shortest path between two users
- Network routing: finding the path with the least hops
- Graph connectivity verification
- Cycle detection (on undirected graphs)

#### Pseudocode

```
BFS(G, start):
    queue = [start]
    visited = {start}
    order = [start]
    parent = {start: None}
    
    while queue not empty:
        node = queue.dequeue()
        
        for neighbor in G.neighbors(node):
            if neighbor not in visited:
                visited.add(neighbor)
                queue.enqueue(neighbor)
                order.append(neighbor)
                parent[neighbor] = node
    
    return visited, order, parent
```

#### Visual Example

For a graph with starting node A:
- Level 0: A
- Level 1: all direct neighbors of A
- Level 2: all neighbors of nodes at level 1 (not already visited)
- etc.

### 4.1 BFS Implementation


In [ ]:
def bfs(G, start, end=None):
    """
    Breadth-first search (BFS)
    
    Parameters:
    - G: NetworkX graph
    - start: starting node
    - end: destination node (optional, if None, traverses entire graph)
    
    Returns:
    - visited: set of visited nodes
    - path: path to 'end' if specified, None otherwise
    - order: order of node visits
    """
    visited = set()
    queue = deque([start])
    visited.add(start)
    order = [start]
    parent = {start: None}
    
    while queue:
        node = queue.popleft()
        
        # If we're looking for a specific node and found it
        if end is not None and node == end:
            # Reconstruct the path
            path = []
            current = end
            while current is not None:
                path.append(current)
                current = parent[current]
            return visited, list(reversed(path)), order
        
        # Traverse neighbors
        for neighbor in G.neighbors(node):
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)
                parent[neighbor] = node
                order.append(neighbor)
    
    # Si end était spécifié mais non trouvé
    if end is not None:
        return visited, None, order
    
    return visited, None, order

# Test du BFS
print("Exécution du BFS...")
visited_bfs, path_bfs, order_bfs = bfs(G, start_node, end_node)

print(f"Nombre de nœuds visités : {len(visited_bfs)}")
print(f"Longueur du chemin trouvé : {len(path_bfs) if path_bfs else 'Aucun chemin'}")

if path_bfs:
    print(f"Chemin (10 premiers nœuds) : {path_bfs[:10]}...")
    print(f"Chemin (10 derniers nœuds) : ...{path_bfs[-10:]}")


### 4.2 BFS Visualization


In [ ]:
# Créer un sous-graphe contenant les nœuds visités par le BFS
if path_bfs:
    nodes_in_path = set()
    for i in range(len(path_bfs) - 1):
        nodes_in_path.add(path_bfs[i])
        nodes_in_path.add(path_bfs[i+1])
        # Ajouter les voisins immédiats pour voir le contexte
        for neighbor in G.neighbors(path_bfs[i]):
            nodes_in_path.add(neighbor)

    G_bfs = G.subgraph(list(nodes_in_path)[:500]).copy()  # Limiter à 500 nœuds pour la performance

    # Créer un dictionnaire de couleurs pour montrer l'ordre de visite
    node_colors = {}
    for i, node in enumerate(order_bfs[:100]):  # Limiter pour la performance
        # Utiliser une palette de couleurs
        if node in path_bfs:
            node_colors[node] = 'red'
        else:
            # Couleur basée sur l'ordre de visite
            intensity = int(255 * (i / min(100, len(order_bfs))))
            node_colors[node] = f'rgb({intensity}, {255-intensity}, 200)'

    fig = plot_graph_plotly(
        G_bfs, 
        title="Parcours BFS - Réseau routier de Paris",
        highlight_nodes=[start_node, end_node],
        highlight_path=path_bfs,
        node_colors=node_colors
    )
    fig.show()

    print(f"Visualisation du BFS : {len(G_bfs.nodes())} nœuds affichés")
else:
    print("Aucun chemin trouvé avec BFS, impossible de visualiser")


### 4.3 BFS with NetworkX (built-in function)


In [ ]:
# Utilisation de la fonction BFS intégrée de NetworkX
try:
    bfs_edges_nx = list(nx.bfs_edges(G, source=start_node))
    bfs_tree_nx = nx.bfs_tree(G, source=start_node)
    
    print(f"BFS NetworkX - Nombre d'arêtes explorées : {len(bfs_edges_nx)}")
    print(f"BFS NetworkX - Taille de l'arbre : {len(bfs_tree_nx.nodes())} nœuds")
    
    # Chemin le plus court avec BFS (si les poids sont égaux)
    try:
        path_bfs_nx = nx.shortest_path(G, source=start_node, target=end_node, method='bfs')
        print(f"Chemin BFS NetworkX - Longueur : {len(path_bfs_nx)}")
    except nx.NetworkXNoPath:
        print("Aucun chemin trouvé avec BFS NetworkX")
        
except Exception as e:
    print(f"Erreur avec BFS NetworkX : {e}")


## 5. Depth-First Search (DFS)

### 5.0 DFS Theory

**Depth-First Search (DFS)** is a graph traversal algorithm that explores the graph by going as far as possible along a branch before backtracking.

#### Principle of Operation

DFS uses a **stack (LIFO: Last In First Out)** or **recursion** to maintain the order of visits:

1. **Initialization**: Start with a source node, mark it as visited
2. **Recursive exploration**: For each visited node:
   - Recursively explore the first unvisited neighbor
   - Continue until reaching a node with no unvisited neighbors
   - Backtrack to explore other branches
3. **Result**: We obtain a depth-first traversal of the graph

#### Two Main Implementations

1. **Recursive DFS**: Uses the function call stack
2. **Iterative DFS**: Explicitly uses a data stack

#### Important Properties

- **Shortest path**: DFS **does NOT guarantee** finding the shortest path
- **Data structure**: Stack (LIFO) - the last node added is the first to be processed
- **Time complexity**: O(V + E) where V is the number of nodes and E is the number of edges
- **Space complexity**: O(V) for the stack (recursion or explicit stack) and visited nodes

#### Applications

- Cycle detection in a graph
- Topological sort (on directed acyclic graphs)
- Backtracking problem solving (mazes, sudoku)
- Finding strongly connected components
- Tree traversal (preorder, postorder)
- Graph algorithms (Hamiltonian path, etc.)

#### Pseudocode (recursive)

```
DFS_recursive(G, node, visited):
    visited.add(node)
    
    for neighbor in G.neighbors(node):
        if neighbor not in visited:
            DFS_recursive(G, neighbor, visited)
```

#### Pseudocode (iterative with stack)

```
DFS_iterative(G, start):
    stack = [start]
    visited = set()
    
    while stack not empty:
        node = stack.pop()
        
        if node not in visited:
            visited.add(node)
            
            for neighbor in G.neighbors(node):
                if neighbor not in visited:
                    stack.push(neighbor)
    
    return visited
```

#### Visual Example

For a graph with starting node A having neighbors B and C:
- Visit A
- Choose B and recursively explore its entire branch before returning
- Return to A and explore C and its branch

### 5.1 DFS Implementation (recursive)


In [ ]:
def dfs_recursive(G, node, visited, order, end=None, parent=None, path=None):
    """
    DFS récursif
    
    Parameters:
    - G: graphe NetworkX
    - node: nœud courant
    - visited: ensemble des nœuds visités
    - order: liste de l'ordre de visite
    - end: nœud d'arrivée (optionnel)
    - parent: dictionnaire parent pour reconstruire le chemin
    - path: chemin actuel (pour retourner le chemin trouvé)
    """
    visited.add(node)
    order.append(node)
    
    if path is None:
        path = []
    path.append(node)
    
    # Si on a trouvé le nœud cible
    if end is not None and node == end:
        return True, path
    
    # Parcourir les voisins
    for neighbor in G.neighbors(node):
        if neighbor not in visited:
            if parent is not None:
                parent[neighbor] = node
            found, found_path = dfs_recursive(G, neighbor, visited, order, end, parent, path.copy())
            if found:
                return True, found_path
    
    return False, None

def dfs(G, start, end=None):
    """
    Parcours en profondeur (DFS)
    
    Parameters:
    - G: graphe NetworkX
    - start: nœud de départ
    - end: nœud d'arrivée (optionnel)
    
    Returns:
    - visited: ensemble des nœuds visités
    - path: chemin vers 'end' si spécifié, None sinon
    - order: ordre de visite des nœuds
    """
    visited = set()
    order = []
    parent = {start: None}
    
    found, path = dfs_recursive(G, start, visited, order, end, parent, None)
    
    if end is not None:
        return visited, path if found else None, order
    
    return visited, None, order

# Test du DFS
print("Exécution du DFS...")
visited_dfs, path_dfs, order_dfs = dfs(G, start_node, end_node)

print(f"Nombre de nœuds visités : {len(visited_dfs)}")
print(f"Longueur du chemin trouvé : {len(path_dfs) if path_dfs else 'Aucun chemin'}")

if path_dfs:
    print(f"Chemin (10 premiers nœuds) : {path_dfs[:10]}...")
    print(f"Chemin (10 derniers nœuds) : ...{path_dfs[-10:]}")


### 5.2 Implémentation du DFS (itératif avec pile)


In [ ]:
def dfs_iterative(G, start, end=None):
    """
    DFS itératif utilisant une pile
    
    Parameters:
    - G: graphe NetworkX
    - start: nœud de départ
    - end: nœud d'arrivée (optionnel)
    
    Returns:
    - visited: ensemble des nœuds visités
    - path: chemin vers 'end' si spécifié, None sinon
    - order: ordre de visite des nœuds
    """
    visited = set()
    stack = [start]
    order = []
    parent = {start: None}
    
    while stack:
        node = stack.pop()
        
        if node not in visited:
            visited.add(node)
            order.append(node)
            
            # Si on a trouvé le nœud cible
            if end is not None and node == end:
                # Reconstruire le chemin
                path = []
                current = end
                while current is not None:
                    path.append(current)
                    current = parent[current]
                return visited, list(reversed(path)), order
            
            # Ajouter les voisins à la pile (en ordre inverse pour cohérence)
            neighbors = list(G.neighbors(node))
            neighbors.reverse()
            for neighbor in neighbors:
                if neighbor not in visited:
                    stack.append(neighbor)
                    if neighbor not in parent:
                        parent[neighbor] = node
    
    # Si end était spécifié mais non trouvé
    if end is not None:
        return visited, None, order
    
    return visited, None, order

# Test du DFS itératif
print("Exécution du DFS itératif...")
visited_dfs_iter, path_dfs_iter, order_dfs_iter = dfs_iterative(G, start_node, end_node)

print(f"Nombre de nœuds visités : {len(visited_dfs_iter)}")
print(f"Longueur du chemin trouvé : {len(path_dfs_iter) if path_dfs_iter else 'Aucun chemin'}")

if path_dfs_iter:
    print(f"Chemin (10 premiers nœuds) : {path_dfs_iter[:10]}...")
    print(f"Chemin (10 derniers nœuds) : ...{path_dfs_iter[-10:]}")


### 5.3 Visualisation du DFS


In [ ]:
# Créer un sous-graphe contenant les nœuds visités par le DFS
if path_dfs_iter:
    nodes_in_path_dfs = set()
    for i in range(len(path_dfs_iter) - 1):
        nodes_in_path_dfs.add(path_dfs_iter[i])
        nodes_in_path_dfs.add(path_dfs_iter[i+1])
        for neighbor in G.neighbors(path_dfs_iter[i]):
            nodes_in_path_dfs.add(neighbor)
    
    G_dfs = G.subgraph(list(nodes_in_path_dfs)[:500]).copy()
    
    # Créer un dictionnaire de couleurs pour montrer l'ordre de visite
    node_colors_dfs = {}
    for i, node in enumerate(order_dfs_iter[:100]):
        if node in path_dfs_iter:
            node_colors_dfs[node] = 'red'
        else:
            intensity = int(255 * (i / min(100, len(order_dfs_iter))))
            node_colors_dfs[node] = f'rgb({intensity}, 200, {255-intensity})'
    
    fig = plot_graph_plotly(
        G_dfs, 
        title="Parcours DFS - Réseau routier de Paris",
        highlight_nodes=[start_node, end_node],
        highlight_path=path_dfs_iter,
        node_colors=node_colors_dfs
    )
    fig.show()
    
    print(f"Visualisation du DFS : {len(G_dfs.nodes())} nœuds affichés")
else:
    print("Aucun chemin trouvé avec DFS")


### 5.4 DFS avec NetworkX (fonction intégrée)


In [ ]:
# Utilisation de la fonction DFS intégrée de NetworkX
try:
    dfs_edges_nx = list(nx.dfs_edges(G, source=start_node))
    dfs_tree_nx = nx.dfs_tree(G, source=start_node)
    
    print(f"DFS NetworkX - Nombre d'arêtes explorées : {len(dfs_edges_nx)}")
    print(f"DFS NetworkX - Taille de l'arbre : {len(dfs_tree_nx.nodes())} nœuds")
    
except Exception as e:
    print(f"Erreur avec DFS NetworkX : {e}")


## 6. Comparaison BFS vs DFS

| Caractéristique | BFS | DFS |
|----------------|-----|-----|
| Structure utilisée | File (FIFO) | Pile (LIFO) |
| Type de parcours | Niveau par niveau | En profondeur |
| Plus court chemin (graphe non pondéré) | Oui | Non garantie |
| Utilisation mémoire | Plus élevée (garde tous les niveaux) | Plus faible |
| Complexité temporelle | O(V + E) | O(V + E) |


In [ ]:
# Comparaison des résultats
print("=== Comparaison BFS vs DFS ===\n")

print(f"BFS:")
print(f"  - Nœuds visités : {len(visited_bfs)}")
print(f"  - Longueur du chemin : {len(path_bfs) if path_bfs else 'N/A'}")

print(f"\nDFS:")
print(f"  - Nœuds visités : {len(visited_dfs_iter)}")
print(f"  - Longueur du chemin : {len(path_dfs_iter) if path_dfs_iter else 'N/A'}")

if path_bfs and path_dfs_iter:
    print(f"\nLe BFS trouve un chemin de {len(path_bfs)} nœuds")
    print(f"Le DFS trouve un chemin de {len(path_dfs_iter)} nœuds")
    print(f"Différence : {abs(len(path_bfs) - len(path_dfs_iter))} nœuds")
    print(f"\nNote : Sur un graphe non pondéré, le BFS garantit le plus court chemin en nombre de nœuds.")


## 7. Algorithmes de Plus Court Chemin

### 7.1 Dijkstra (plus court chemin avec poids)

L'algorithme de Dijkstra trouve le plus court chemin dans un graphe pondéré.


In [ ]:
# Vérifier si le graphe a des attributs de longueur/distance
sample_edge = list(G.edges(data=True))[0]
print(f"Exemple d'arête avec attributs : {sample_edge}")

# Utiliser la longueur si disponible, sinon utiliser une distance calculée
if 'length' in sample_edge[2]:
    weight = 'length'
    print("\nUtilisation de l'attribut 'length' pour les poids")
elif 'weight' in sample_edge[2]:
    weight = 'weight'
    print("\nUtilisation de l'attribut 'weight' pour les poids")
else:
    # Calculer la distance géodésique si lon/lat disponibles
    weight = None
    print("\nCalcul de la distance géodésique pour les poids")
    
    def calculate_distance(u, v):
        """Calcule la distance géodésique entre deux nœuds"""
        try:
            if 'x' in G.nodes[u] and 'y' in G.nodes[u]:
                x1, y1 = G.nodes[u]['x'], G.nodes[u]['y']
                x2, y2 = G.nodes[v]['x'], G.nodes[v]['y']
            elif 'lon' in G.nodes[u] and 'lat' in G.nodes[u]:
                x1, y1 = G.nodes[u]['lon'], G.nodes[u]['lat']
                x2, y2 = G.nodes[v]['lon'], G.nodes[v]['lat']
            else:
                return 1.0  # Distance par défaut
            
            # Distance euclidienne simple (ou utiliser geopy pour distance réelle)
            return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
        except:
            return 1.0
    
    # Ajouter les poids calculés au graphe
    for u, v, d in G.edges(data=True):
        d['calculated_length'] = calculate_distance(u, v)
    weight = 'calculated_length'


In [ ]:
# Calcul du plus court chemin avec Dijkstra
try:
    print("Calcul du plus court chemin avec Dijkstra...")
    path_dijkstra = nx.shortest_path(G, source=start_node, target=end_node, weight=weight, method='dijkstra')
    length_dijkstra = nx.shortest_path_length(G, source=start_node, target=end_node, weight=weight, method='dijkstra')
    
    print(f"Chemin Dijkstra - Longueur : {len(path_dijkstra)} nœuds")
    print(f"Distance totale : {length_dijkstra:.2f} mètres" if weight else f"Distance totale : {length_dijkstra:.2f}")
    print(f"Chemin (10 premiers nœuds) : {path_dijkstra[:10]}...")
    print(f"Chemin (10 derniers nœuds) : ...{path_dijkstra[-10:]}")
    
except Exception as e:
    print(f"Erreur avec Dijkstra : {e}")
    path_dijkstra = None
    length_dijkstra = None


### 7.2 A* (A-star) - Heuristique pour optimisation

#### 7.2.0 Théorie de l'algorithme A*

L'algorithme **A*** (prononcé "A-star") est une amélioration de l'algorithme de Dijkstra qui utilise une **fonction heuristique** pour diriger la recherche vers le but, réduisant ainsi le nombre de nœuds explorés.

#### Principe de fonctionnement

A* combine deux informations pour décider quel nœud explorer ensuite :

1. **g(n)** : Coût réel du chemin depuis le nœud de départ jusqu'au nœud n
2. **h(n)** : Estimation heuristique du coût restant depuis le nœud n jusqu'au but

La fonction d'évaluation est : **f(n) = g(n) + h(n)**

L'algorithme explore toujours le nœud avec la plus petite valeur f(n).

#### Fonction heuristique

Une bonne heuristique doit être :

- **Admissible** : Ne surestime jamais le coût réel vers le but (h(n) ≤ coût réel)
- **Consistante/Monotone** : Pour tout nœud n et son successeur n', h(n) ≤ c(n,n') + h(n')

#### Propriétés importantes

- **Optimalité** : Si l'heuristique est admissible, A* garantit de trouver le chemin optimal
- **Efficacité** : Explore généralement moins de nœuds que Dijkstra, surtout avec une bonne heuristique
- **Complexité temporelle** : O((V + E) log V) dans le pire cas (comme Dijkstra), mais souvent meilleur en pratique
- **Complexité spatiale** : O(V) pour stocker les nœuds explorés

#### Comparaison avec Dijkstra

| Caractéristique | Dijkstra | A* |
|----------------|----------|-----|
| Heuristique | Non | Oui |
| Exploration | Uniforme (en cercles) | Dirigée vers le but |
| Nœuds explorés | Plus nombreux | Moins nombreux (avec bonne heuristique) |
| Optimalité | Oui | Oui (si heuristique admissible) |

#### Heuristiques communes

1. **Distance euclidienne** : h(n) = distance euclidienne entre n et le but (pour graphes géographiques)
2. **Distance de Manhattan** : h(n) = |x_n - x_goal| + |y_n - y_goal| (pour grilles)
3. **Distance de Chebyshev** : Pour mouvements diagonaux autorisés

#### Applications

- Jeux vidéo : pathfinding de personnages
  - Exemples : Starcraft, Age of Empires, jeux de stratégie
- Robotique : planification de trajectoire
- Navigation GPS (variantes avec contraintes)
- Intelligence artificielle : résolution de puzzles, recherche dans des espaces d'états

#### Pseudocode

```
A_star(G, start, goal, heuristic):
    open_set = [(heuristic(start, goal), start)]  # (f, node)
    g_score = {node: ∞ for node in G.nodes}
    g_score[start] = 0
    f_score = {node: ∞ for node in G.nodes}
    f_score[start] = heuristic(start, goal)
    parent = {start: None}
    
    while open_set not empty:
        current = open_set.extract_min()  # nœud avec f le plus petit
        
        if current == goal:
            return reconstruct_path(parent, current)
        
        for neighbor in G.neighbors(current):
            tentative_g = g_score[current] + weight(current, neighbor)
            
            if tentative_g < g_score[neighbor]:
                parent[neighbor] = current
                g_score[neighbor] = tentative_g
                f_score[neighbor] = g_score[neighbor] + heuristic(neighbor, goal)
                open_set.insert_or_update((f_score[neighbor], neighbor))
    
    return None  # Pas de chemin trouvé
```

L'algorithme A* est une amélioration de Dijkstra qui utilise une heuristique pour diriger la recherche.


In [ ]:
# Fonction heuristique pour A* (distance euclidienne vers le but)
def heuristic(u, v):
    """Heuristique : distance euclidienne entre deux nœuds"""
    try:
        if 'x' in G.nodes[u] and 'y' in G.nodes[u]:
            x1, y1 = G.nodes[u]['x'], G.nodes[u]['y']
            x2, y2 = G.nodes[v]['x'], G.nodes[v]['y']
        elif 'lon' in G.nodes[u] and 'lat' in G.nodes[u]:
            x1, y1 = G.nodes[u]['lon'], G.nodes[u]['lat']
            x2, y2 = G.nodes[v]['lon'], G.nodes[v]['lat']
        else:
            return 0
        
        return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    except:
        return 0

# Calcul du plus court chemin avec A*
try:
    print("Calcul du plus court chemin avec A*...")
    path_astar = nx.astar_path(G, source=start_node, target=end_node, weight=weight, heuristic=heuristic)
    length_astar = nx.astar_path_length(G, source=start_node, target=end_node, weight=weight, heuristic=heuristic)
    
    print(f"Chemin A* - Longueur : {len(path_astar)} nœuds")
    print(f"Distance totale : {length_astar:.2f} mètres" if weight else f"Distance totale : {length_astar:.2f}")
    print(f"Chemin (10 premiers nœuds) : {path_astar[:10]}...")
    print(f"Chemin (10 derniers nœuds) : ...{path_astar[-10:]}")
    
except Exception as e:
    print(f"Erreur avec A* : {e}")
    path_astar = None
    length_astar = None


### 7.3 Comparaison des algorithmes de plus court chemin


In [ ]:
import time

print("=== Comparaison des algorithmes ===\n")

# Construire les algorithmes en fonction de la disponibilité des poids
algorithms = {
    'BFS (non pondéré)': lambda: nx.shortest_path(G, start_node, end_node, method='bfs')
}

if weight:
    algorithms['Dijkstra'] = lambda: nx.shortest_path(G, start_node, end_node, weight=weight, method='dijkstra')
    algorithms['A*'] = lambda: nx.astar_path(G, start_node, end_node, weight=weight, heuristic=heuristic)
else:
    # Si pas de poids, Dijkstra est équivalent à BFS
    algorithms['Dijkstra (sans poids)'] = lambda: nx.shortest_path(G, start_node, end_node, method='dijkstra')

results = {}
for name, func in algorithms.items():
    try:
        start_time = time.time()
        path = func()
        elapsed = time.time() - start_time
        length = len(path)
        
        if weight:
            path_length = nx.shortest_path_length(G, start_node, end_node, weight=weight)
            results[name] = {
                'path': path,
                'node_count': length,
                'distance': path_length,
                'time': elapsed
            }
        else:
            results[name] = {
                'path': path,
                'node_count': length,
                'distance': None,
                'time': elapsed
            }
        print(f"{name}:")
        print(f"  - Temps : {elapsed*1000:.2f} ms")
        print(f"  - Nombre de nœuds : {length}")
        if weight:
            print(f"  - Distance : {path_length:.2f}")
        print()
    except Exception as e:
        print(f"{name}: Erreur - {e}\n")


### 7.4 Visualisation des plus courts chemins


In [ ]:
# Créer une visualisation comparant les différents chemins
if path_dijkstra:
    # Créer un sous-graphe contenant les chemins
    all_path_nodes = set()
    for path in [path_bfs, path_dfs_iter, path_dijkstra, path_astar]:
        if path:
            all_path_nodes.update(path)
            # Ajouter quelques voisins pour le contexte
            for node in path[:50]:  # Limiter pour la performance
                for neighbor in list(G.neighbors(node))[:3]:
                    all_path_nodes.add(neighbor)
    
    G_paths = G.subgraph(list(all_path_nodes)[:1000]).copy()
    
    # Visualisation avec différents chemins
    fig = go.Figure()
    
    # Récupération des coordonnées (longitude, latitude)
    pos = {}
    for node in G_paths.nodes():
        if 'x' in G_paths.nodes[node] and 'y' in G_paths.nodes[node]:
            pos[node] = (G_paths.nodes[node]['x'], G_paths.nodes[node]['y'])
        elif 'lon' in G_paths.nodes[node] and 'lat' in G_paths.nodes[node]:
            pos[node] = (G_paths.nodes[node]['lon'], G_paths.nodes[node]['lat'])
        else:
            pos[node] = (node, 0)
    
    # Calcul du centre de la carte basé sur les chemins
    path_coords = []
    for path in [path_dijkstra, path_astar, path_bfs]:
        if path:
            for node in path:
                if node in pos and isinstance(pos[node][0], (int, float)) and isinstance(pos[node][1], (int, float)):
                    path_coords.append(pos[node])
    
    if path_coords:
        center_lon = sum(c[0] for c in path_coords) / len(path_coords)
        center_lat = sum(c[1] for c in path_coords) / len(path_coords)
    else:
        center_lon, center_lat = 2.3522, 48.8566  # Paris par défaut
    
    # Arêtes du graphe - groupées en une seule trace pour performance
    edge_lons = []
    edge_lats = []
    for edge in G_paths.edges():
        if edge[0] in pos and edge[1] in pos:
            lon0, lat0 = pos[edge[0]]
            lon1, lat1 = pos[edge[1]]
            if (isinstance(lon0, (int, float)) and isinstance(lat0, (int, float)) and
                isinstance(lon1, (int, float)) and isinstance(lat1, (int, float))):
                edge_lons.extend([lon0, lon1, None])
                edge_lats.extend([lat0, lat1, None])
    
    if edge_lons and edge_lats:
        fig.add_trace(go.Scattermap(
            mode='lines',
            lon=edge_lons,
            lat=edge_lats,
            line=dict(width=0.5, color='lightgray'),
            hoverinfo='none',
            name='Réseau routier',
            showlegend=True,
            legendgroup='network'
        ))
    
    # Chemin Dijkstra
    if path_dijkstra:
        path_lons = [pos[node][0] for node in path_dijkstra if node in pos and isinstance(pos[node][0], (int, float))]
        path_lats = [pos[node][1] for node in path_dijkstra if node in pos and isinstance(pos[node][1], (int, float))]
        if path_lons and path_lats:
            fig.add_trace(go.Scattermap(
                mode='lines+markers',
                lon=path_lons,
                lat=path_lats,
                line=dict(width=4, color='blue'),
                marker=dict(size=6, color='blue', opacity=0.7),
                name=f'Dijkstra ({len(path_dijkstra)} nœuds)',
                showlegend=True,
                legendgroup='paths'
            ))
    
    # Chemin A*
    if path_astar:
        path_lons = [pos[node][0] for node in path_astar if node in pos and isinstance(pos[node][0], (int, float))]
        path_lats = [pos[node][1] for node in path_astar if node in pos and isinstance(pos[node][1], (int, float))]
        if path_lons and path_lats:
            fig.add_trace(go.Scattermap(
                mode='lines+markers',
                lon=path_lons,
                lat=path_lats,
                line=dict(width=4, color='green'),
                marker=dict(size=6, color='green', opacity=0.7),
                name=f'A* ({len(path_astar)} nœuds)',
                showlegend=True,
                legendgroup='paths'
            ))
    
    # Chemin BFS
    if path_bfs:
        path_lons = [pos[node][0] for node in path_bfs if node in pos and isinstance(pos[node][0], (int, float))]
        path_lats = [pos[node][1] for node in path_bfs if node in pos and isinstance(pos[node][1], (int, float))]
        if path_lons and path_lats:
            fig.add_trace(go.Scattermap(
                mode='lines+markers',
                lon=path_lons,
                lat=path_lats,
                line=dict(width=3, color='red'),
                marker=dict(size=5, color='red', opacity=0.7),
                name=f'BFS ({len(path_bfs)} nœuds)',
                showlegend=True,
                legendgroup='paths'
            ))
    
    # Points de départ et d'arrivée
    if start_node in pos and end_node in pos:
        start_lon, start_lat = pos[start_node]
        end_lon, end_lat = pos[end_node]
        if (isinstance(start_lon, (int, float)) and isinstance(start_lat, (int, float)) and
            isinstance(end_lon, (int, float)) and isinstance(end_lat, (int, float))):
            fig.add_trace(go.Scattermap(
                mode='markers',
                lon=[start_lon],
                lat=[start_lat],
                marker=dict(size=18, color='orange'),
                name='Départ',
                showlegend=True,
                legendgroup='points'
            ))
            fig.add_trace(go.Scattermap(
                mode='markers',
                lon=[end_lon],
                lat=[end_lat],
                marker=dict(size=18, color='purple'),
                name='Arrivée',
                showlegend=True,
                legendgroup='points'
            ))
    
    fig.update_layout(
        title="Comparaison des Plus Courts Chemins - Paris",
        showlegend=True,
        mapbox=dict(
            style="open-street-map",
            center=dict(lon=center_lon, lat=center_lat),
            zoom=12,
            bearing=0,
            pitch=0
        ),
        margin=dict(l=0, r=0, t=40, b=0),
        height=800
    )
    
    fig.show()


## 9. Résumé

### Points clés à retenir :

1. **BFS (Breadth-First Search)** :
   - Explore niveau par niveau
   - Garantit le plus court chemin en nombre de nœuds (graphe non pondéré)
   - Utilise une file (FIFO)

2. **DFS (Depth-First Search)** :
   - Explore en profondeur avant de revenir en arrière
   - Ne garantit pas le plus court chemin
   - Utilise une pile (LIFO) ou récursivité

3. **Dijkstra** :
   - Trouve le plus court chemin dans un graphe pondéré
   - Nécessite des poids positifs
   - Complexité : O((V + E) log V) avec une bonne implémentation

4. **A*** :
   - Amélioration de Dijkstra avec heuristique
   - Plus rapide que Dijkstra dans la plupart des cas
   - Optimal si l'heuristique est admissible (ne surestime jamais)

### Applications pratiques :
- Navigation GPS (routage)
- Réseaux sociaux (recherche de connexions)
- Intelligence artificielle (jeux, planification)
- Analyse de réseaux (Internet, transport)
